In [48]:
import mysql.connector
import yaml
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)

with open('config.yaml', 'r') as c:
    config = yaml.load(c)
cnx = mysql.connector.connect(user=config["database"]["user"],
                              password=config["database"]["password"],
                              host=config["database"]["host"],
                              database=config["database"]["db"])
cursor = cnx.cursor()

def execute_query(q):
    cursor.execute(q)
    return cursor.fetchall()

def plot_bar_chart(rows):
    x = []
    y = []
    for row in rows:
        x.append('{}-{}'.format(row[0], row[1]))
        y.append(row[2])
    data = [go.Bar(x=x, y=y)]
    iplot(data)

In [39]:
query_comments_by_month = (
    'SELECT year(created_time) year, month(created_time) month, count(*) count '
    'FROM comment '
    'WHERE page=1 AND year(created_time)>=2011 '
    'GROUP BY year(created_time), month(created_time)'
)
rows_comments_by_month = execute_query(query_comments_by_month)
plot_bar_chart(rows_comments_by_month)

In [46]:
query_comments_by_month = (
    'SELECT year(created_time) year, month(created_time) month, count(*) count '
    'FROM comment '
    'WHERE page NOT IN (11, 12, 13, 20) AND year(created_time)>=2011 '
    'GROUP BY year(created_time), month(created_time)'
)
rows_comments_by_month = execute_query(query_comments_by_month)
plot_bar_chart(rows_comments_by_month)

In [41]:
query_posts_by_month = (
    'SELECT year(created_time) year, month(created_time) month, count(*) count '
    'FROM post '
    'WHERE page=1 AND year(created_time)>=2011 '
    'GROUP BY year(created_time), month(created_time)'
)
rows_posts_by_month = execute_query(query_posts_by_month)
plot_bar_chart(rows_posts_by_month)

In [42]:
rows_comments_per_post_by_month = []
for (comments, posts) in zip(rows_comments_by_month, rows_posts_by_month):
    rows_comments_per_post_by_month.append((comments[0], comments[1], comments[2] / posts[2]))
plot_bar_chart(rows_comments_per_post_by_month)

In [38]:
rows_comments_by_month[:5], rows_posts_by_month[:5]

([(2010, 3, 1), (2010, 4, 14), (2010, 5, 253), (2010, 6, 98), (2010, 7, 98)],
 [(2009, 8, 1), (2010, 1, 1), (2010, 2, 1), (2010, 3, 1), (2010, 4, 7)])

In [52]:
query_all_comments_by_month = (
    'SELECT year(created_time) year, month(created_time) month, count(*) count '
    'FROM comment '
    'GROUP BY year(created_time), month(created_time)'
)
rows_all_comments_by_month = execute_query(query_all_comments_by_month)
plot_bar_chart(rows_all_comments_by_month)